In [ ]:
# Install Prophet
!pip install prophet

import pandas as pd
from prophet import Prophet
import zipfile

# Step 1: Load dataset
df = pd.read_csv("Sample - Superstore.csv", encoding='latin1')

# Step 2: Prepare monthly sales data
df['Order Date'] = pd.to_datetime(df['Order Date'])
monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({
    'Sales':'sum'
}).reset_index()

# Prophet requires 'ds' and 'y'
df_prophet = monthly_sales.rename(columns={'Order Date':'ds', 'Sales':'y'})

# Step 3: Train Prophet Model
model = Prophet(interval_width=0.95)
model.fit(df_prophet)

# Step 4: Forecast next 12 months
future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

# Step 5: Merge actual + forecast
final = forecast[['ds','yhat','yhat_lower','yhat_upper']].merge(
    df_prophet, on='ds', how='left'
)

# Step 6: Save CSVs
final.to_csv("sales_forecast.csv", index=False)

top_products = df.groupby("Product Name").agg({"Sales":"sum"}).reset_index()
top_products = top_products.sort_values("Sales", ascending=False).head(10)
top_products.to_csv("top_products.csv", index=False)

df.to_csv("superstore_clean.csv", index=False)

# Step 7: Zip all files
with zipfile.ZipFile("dashboard_data_csv.zip","w") as z:
    z.write("sales_forecast.csv")
    z.write("top_products.csv")
    z.write("superstore_clean.csv")

print("✅ Done! Files generated: sales_forecast.csv, top_products.csv, superstore_clean.csv (all zipped)")


/tmp/ipython-input-772444598.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgkw38lfz/0w8sj6m3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpgkw38lfz/gorqy8uc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=63352', 'data', 'file=/tmp/tmpgkw38lfz/0w8sj6m3.json', 'init=/tmp/tmpgkw38lfz/gorqy8uc.json', 'output', 'file=/tmp/tmpgkw38lfz/prophet_modelfoo41wzh/prophet_model-20250831063520.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
06:35:20 - cmdstanpy - IN

✅ Done! Files generated: sales_forecast.csv, top_products.csv, superstore_clean.csv (all zipped)
